<div dir="rtl">
  <center>
    <img src="https://user-images.githubusercontent.com/7662492/229289746-89c5a4c7-afa6-4d46-a0e6-63dfdeb98285.jpg" width="250px"/>

  <h1>تفريغ - الكتب المُيسّرة</h1>
  </center>

  <p>مرحبًا بك في تفريغ لتفريغ المواد الصوتية والمرئية باستخدام تقنيات الذكاء الاصطناعي. لاستخدام تفريغ:</p>

  <ol>
    <li>قم بإدخال روابط المواد المطلوب تحويلها من منصة YouTube أو أي منصة أخرى في حقل "urls" وتأكد من فصلها بمسافة، أو قم بترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</li>
    <li>(اختياري) قم بتحديد أقل عدد من الكلمات في كل جزء من أجزاء التفريغ. يؤثر هذا في طول الأجزاء التي سيتم تفريغها</li>
    <li>
      إذا كنت تريد استخدام مجموعة نماذج Whisper:
      <ul>
        <li>قم باختيار النموذج المُراد استخدامه في حقل "model"</li>
        <li>قم باختيار لغة المادة في حقل "language"</li>
        <li>قم باختيار المهمة المُراد إنجازها في حقل "task" سواء كانت تفريغ المادة أو ترجمتها للانجليزية</li>
      </ul>
    </li>
    <li>
      إذا كنت تريد استخدام تقنية wit.ai:
      <ul>
        <li>قم بوضع مفتاح wit.ai الخاص بك في حقل "wit_api_key"</li>
        <li>(اختياري) قم بتحديد أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف المخرجات</li>
      </ul>
    </li>
    <li>قم بتشغيل الخلية في الأسفل من خلال الضغط على الدائرة المحتوية على السهم</li>
  </ol>

  <p>يمكنك تجربة التفريغ باستخدام نماذج Whisper وتقنية wit.ai واستخدام التفريغ الأفضل لحالتك. كملاحظة عامة، نماذج Whisper تقوم بتفريغ الهمزات والتشكيلات وعلامات الترقيم بشكل أفضل من wit.ai، لكن wit.ai يُنتج أخطاء إملائية أقل.</p>

  <p>عندما ينتهي التحويل سيتم تنزيل الملفات النصية بشكل تلقائي بصيغة <code>txt</code> و <code>srt</code> وسيكون اسم الملف هو مُعرّف المادة على منصة YouTube الذي يكون في آخر رابط المادة: https://youtu.be/<strong>4h5P7jXvW98</strong>.</p>

  <hr>

  <p>يمكنك متابعة مشروع <strong>الكتب المُيسّرة</strong> والتواصل معنا من خلال:</p>

  <ul>
    <li><a href="https://t.me/ieasybooks">قناتنا على تيليجرام</a></li>
    <li><a href="https://www.youtube.com/@ieasybooks">قناتنا على يوتيوب</a></li>
    <li><a href="https://twitter.com/iieasybooks">حسابنا على تويتر</a></li>
    <li><a href="https://www.facebook.com/ieasybooks">صفحتنا على فيسبوك</a></li>
    <li><a href="https://github.com/ieasybooks">حسابنا على GitHub (للمبرمجين)</a></li>
    <li>بريدنا الالكتروني: easybooksdev@gmail.com</li>
  </ul>
</div>

In [ ]:
# @title <h1><center>تفريغ</center></h1>

print('جارٍ تجهيز بيئة العمل.')

# Setup Tafrigh.
!apt-get install -q portaudio19-dev python3-pyaudio > pyaudio_fix_logs.txt
%pip install -U tafrigh[wit,whisper]==1.7.6 > install_logs.txt
%pip install -U yt-dlp["default"] > update_yt_dlp_logs.txt

# Start: Quick fix related to Colab, HuggingFace, and faster-whisper.
!apt install libcublas11 > fix_logs.txt
!pip install ctranslate2==4.5.0 > ctranslate2_downgrade_logs.txt

!apt install nodejs -y

from huggingface_hub.utils import _runtime
_runtime._is_google_colab = False
# End: Quick fix related to Colab, HuggingFace, and faster-whisper.

# Get inputs.

# @markdown <h1 dir="rtl">مدخلات عامة</h1>

# @markdown <p dir="rtl">روابط المواد المطلوب تفريغها وتأكد من فصلها بمسافة، أو قم بترك الحقل فارغًا لتفريغ المواد التي قمت برفعها</p>
urls = 'https://www.youtube.com/playlist?list=PLSSxr3Rf2_X2oKwiy4UhzIdj4ACzB6dee'  # @param { type: "string" }

# @markdown <p dir="rtl">أقل عدد من الكلمات في كل جزء من أجزاء التفريغ</p>
min_words_per_segment = 1  # @param {type:"slider", min:1, max:100, step:1}

# @markdown <p dir="rtl">صيغة الملفات المطلوبة</p>
output_format = "TXT فقط"  # @param ["TXT فقط", "SRT فقط", "TXT و SRT"]

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بنماذج Whisper</h1>

# @markdown <p dir="rtl">النموذج المُراد استخدامه للتفريغ</p>
model = 'large-v3 (أفضل دقة)'  # @param ["large-v3 (أفضل دقة)", "medium", "base", "small", "tiny (أقل دقة)"]

# @markdown <p dir="rtl">(اختياري) لغة المادة</p>
language = 'ar'  # @param ["ar", "af", "am", "as", "az", "ba", "be", "bg", "bn", "bo", "br", "bs", "ca", "cs", "cy", "da", "de", "el", "en", "es", "et", "eu", "fa", "fi", "fo", "fr", "gl", "gu", "ha", "haw", "he", "hi", "hr", "ht", "hu", "hy", "id", "is", "it", "ja", "jw", "ka", "kk", "km", "kn", "ko", "la", "lb", "ln", "lo", "lt", "lv", "mg", "mi", "mk", "ml", "mn", "mr", "ms", "mt", "my", "ne", "nl", "nn", "no", "oc", "pa", "pl", "ps", "pt", "ro", "ru", "sa", "sd", "si", "sk", "sl", "sn", "so", "sq", "sr", "su", "sv", "sw", "ta", "te", "tg", "th", "tk", "tl", "tr", "tt", "uk", "ur", "uz", "vi", "yi", "yo", "zh"]

# @markdown <p dir="rtl">(اختياري) المهمة</p>
task = 'تفريغ'  # @param ["تفريغ", "ترجمة"]

# @markdown ---

# @markdown <h1 dir="rtl">مدخلات خاصة بتقنية wit.ai</h1>

# @markdown <p dir="rtl">المفتاح الخاص بك على موقع wit.ai</p>
wit_api_key = ''  # @param { type: "string" }

# @markdown <p dir="rtl">(اختياري) أقصى مدة للتقطيع والتي ستؤثر على طول الجمل في ملف SRT</p>
max_cutting_duration = 15  # @param {type:"slider", min:1, max:17, step:1}

# @markdown ---

# @markdown <h1 dir="rtl">خيارات Google Drive</h1>

# @markdown <p dir="rtl">حفظ الملفات على Google Drive (موصى به لقوائم التشغيل الكبيرة)</p>
use_google_drive = True  # @param {type:"boolean"}

# @markdown <p dir="rtl">اسم المجلد على Google Drive</p>
drive_folder_name = "Tafrigh_Transcripts"  # @param {type:"string"}

if model == 'large-v3 (أفضل دقة)':
    model = 'large-v3'
elif model == 'tiny (أقل دقة)':
    model = 'tiny'

if task == 'تفريغ':
  task = 'transcribe'
elif task == 'ترجمة':
  task = 'translate'

# Parse output format
output_formats = []
if output_format == "TXT فقط":
    output_formats = ['txt']
elif output_format == "SRT فقط":
    output_formats = ['srt']
else:  # "TXT و SRT"
    output_formats = ['txt', 'srt']

# Imports.
import glob
import os
import re
import yt_dlp
import shutil
import zipfile
from datetime import datetime

from google.colab import files
from tafrigh import farrigh, Config

# --- Mount Google Drive if enabled ---
drive_output_dir = None
if use_google_drive:
    print('📁 جارٍ ربط Google Drive...')
    from google.colab import drive
    drive.mount('/content/drive')

    # Create output folder in Drive with timestamp to avoid conflicts
    # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    drive_output_dir = f"/content/drive/MyDrive/{drive_folder_name}"
    if not os.path.exists(drive_output_dir):
        os.makedirs(drive_output_dir, exist_ok=True)
        print(f'✅ تم إنشاء مجلد على Drive: {drive_folder_name}')
    print(f'📍 المسار: {drive_output_dir}\n')

# Setup local directories.
output_dir = 'output'
if not os.path.exists(output_dir):
  os.mkdir(output_dir)
else:
  # Clean up before start
  shutil.rmtree(output_dir)
  os.mkdir(output_dir)

# --- Cookies auto-detection ---
COOKIES_FILE = "cookies.txt"
USE_COOKIES = os.path.exists(COOKIES_FILE)

# --- Start: Fetch video titles ---
print('جارٍ محاولة جلب عناوين الفيديو (اختياري).')

ydl_opts = {
    'quiet': True,
    'no_warnings': True,
    'extract_flat': 'in_playlist',
    'extractor_args': {
        'youtube': {
            'player_client': ['android', 'web'],
            'skip': ['hls', 'dash']
        }
    },
}

# Use cookies if exist
if USE_COOKIES:
    print('الملف cookies.txt موجود، جارٍ استخدامه.')
    ydl_opts.update({
        'cookies': 'cookies.txt',
        'js_runtime': 'node',
    })
else:
    print('الملف cookies.txt غير موجود، سيتم المتابعة بدونه.')

urls_list = [u.strip() for u in urls.split(' ') if u.strip()]
video_titles = {}
for url in urls_list:
  try:
      with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          info_dict = ydl.extract_info(url, download=False)
          if info_dict:
              # Check if it's a playlist
              if info_dict.get('_type') == 'playlist':
                  print(f"✓ اكتشاف قائمة تشغيل: {info_dict.get('title', 'بدون عنوان')}")
                  entries = info_dict.get('entries', [])
                  print(f"  عدد الفيديوهات: {len(entries)}")

                  for entry in entries:
                      if entry:
                          video_id = entry.get('id')
                          video_title = entry.get('title')
                          if video_id and video_title:
                              video_titles[video_id] = video_title
                              print(f"  ✓ {video_title}")
              else:
                  # Single video
                  video_id = info_dict.get('id')
                  video_title = info_dict.get('title')
                  if video_id and video_title:
                      video_titles[video_id] = video_title
                      print(f"✓ تم جلب عنوان: {video_title}")
                  else:
                      print(f"⚠ تعذر الحصول على معلومات كاملة للفيديو")
  except Exception as e:
      print(f"⚠ تحذير: تعذر جلب عنوان الفيديو")
      print(f"   الخطأ: {str(e)[:100]}")
      print(f"   سيتم استخدام معرف الفيديو كاسم للملف")

print(f'\n✓ تم جلب {len(video_titles)} عنوان بنجاح.\n')

# Start Tafrigh.
if wit_api_key:
  print('جارٍ تحويل المواد إلى نصوص باستخدام تقنيات wit.ai.')
else:
  print('جارٍ تحويل المواد إلى نصوص باستخدام نماذج Whisper.')

config = Config(
  input=Config.Input(
    urls_or_paths=urls_list if urls_list else ['.'],
    skip_if_output_exist=False,
    download_retries=3,
    yt_dlp_options=(
      '''
      {
        "cookies": "cookies.txt",
        "js_runtime": "node"
      }
      ''' if USE_COOKIES else '{}'
    ),
    verbose=False,
  ),
  whisper=Config.Whisper(
    model_name_or_path=model,
    task=task,
    language=language,
    use_faster_whisper=True,
    beam_size=5,
    ct2_compute_type='default',
  ),
  wit=Config.Wit(
    wit_client_access_tokens=None if len(wit_api_key.strip()) == 0 else wit_api_key.split(),
    max_cutting_duration=max_cutting_duration,
  ),
  output=Config.Output(
    min_words_per_segment=min_words_per_segment,
    save_files_before_compact=False,
    save_yt_dlp_responses=False,
    output_sample=0,
    output_formats=output_formats,
    output_dir=output_dir,
  ),
)

def get_output_files():
    """Get set of current output files (excluding archive.txt and mp3 files)."""
    extensions = []
    if 'txt' in output_formats:
        extensions.append(f"{output_dir}/*.txt")
    if 'srt' in output_formats:
        extensions.append(f"{output_dir}/*.srt")

    all_files = set()
    for ext_pattern in extensions:
        all_files.update(glob.glob(ext_pattern))

    archive_path = os.path.join(output_dir, 'archive.txt')
    return {f for f in all_files if f != archive_path}

def copy_files_to_drive(new_files, video_titles, drive_output_dir, video_number):
    """Rename files and copy to Google Drive."""
    if not new_files or not drive_output_dir:
        return []

    copied_files = []
    video_title = None

    for file_path in new_files:
        file_name = os.path.basename(file_path)
        base, ext = os.path.splitext(file_name)

        # Determine new name
        if base in video_titles:
            title = video_titles[base]
            video_title = title
            safe_title = re.sub(r'[\\/:*?"<>|.]', '', title).strip()
            if safe_title:
                new_name = f"{safe_title}{ext}"
            else:
                new_name = file_name
        else:
            new_name = file_name

        # Handle duplicate names
        drive_path = os.path.join(drive_output_dir, new_name)
        counter = 1
        base_name, ext = os.path.splitext(new_name)
        while os.path.exists(drive_path):
            drive_path = os.path.join(drive_output_dir, f"{base_name}_{counter}{ext}")
            counter += 1

        try:
            shutil.copy2(file_path, drive_path)
            copied_files.append(drive_path)
        except Exception as e:
            print(f"⚠ خطأ في نسخ الملف: {e}")

    return copied_files

# Track progress and save after each video
if use_google_drive:
    print("⏳ جارٍ معالجة الفيديوهات (سيتم الحفظ على Google Drive بعد كل فيديو)...\n")
else:
    print("⏳ جارٍ معالجة الفيديوهات...\n")

last_video_key = None
video_count = 0
completed_videos = set()
known_files = get_output_files()
all_drive_files = []

try:
    for item in farrigh(config):
        progress_info = item if isinstance(item, dict) else (item[0] if isinstance(item, tuple) else {})

        inner_current = progress_info.get('inner_current', 0)
        outer_current = progress_info.get('outer_current', 0)
        inner_status = progress_info.get('inner_status', '')
        inner_total = progress_info.get('inner_total', 0)

        current_video_key = (outer_current, inner_current)

        # Show progress when starting a new video
        if current_video_key != last_video_key and inner_current > 0:
            if inner_total > 0:
                print(f"📹 معالجة الفيديو {inner_current}/{inner_total}...")
            last_video_key = current_video_key

        # When video completed, save files immediately
        if inner_status == 'completed' and current_video_key not in completed_videos:
            completed_videos.add(current_video_key)
            video_count += 1

            # Get new files for this video
            current_files = get_output_files()
            new_files = current_files - known_files

            if new_files:
                print(f"✅ اكتمل الفيديو {video_count}/{inner_total if inner_total > 0 else '?'}")

                if use_google_drive and drive_output_dir:
                    # Copy to Google Drive
                    copied = copy_files_to_drive(new_files, video_titles, drive_output_dir, video_count)
                    all_drive_files.extend(copied)

                    if copied:
                        file_names = [os.path.basename(f) for f in copied]
                        print(f"   ? تم الحفظ على Drive: {', '.join(file_names)}\n")

            known_files = current_files

    print('\n✅ اكتمل تفريغ جميع الفيديوهات!')

except Exception as e:
    print(f"\n❌ التفريغ فشل: {e}")
    import traceback
    traceback.print_exc()

    if use_google_drive and drive_output_dir:
        print(f"\n⚠️ ملاحظة: الملفات المحفوظة متاحة على Google Drive:")
        print(f"   📁 {drive_output_dir}")

# Final: Create ZIP and download
if use_google_drive and drive_output_dir:
    # Check what files are in Drive folder
    drive_files = []
    if 'txt' in output_formats:
        drive_files.extend(glob.glob(f"{drive_output_dir}/*.txt"))
    if 'srt' in output_formats:
        drive_files.extend(glob.glob(f"{drive_output_dir}/*.srt"))

    if drive_files:
        print(f'\n📁 تم حفظ {len(drive_files)} ملفات على Google Drive')
        print(f'   المسار: {drive_output_dir}')

        # Create final ZIP in Drive
        zip_name = os.path.basename(drive_output_dir) + ".zip"
        zip_path = os.path.join(os.path.dirname(drive_output_dir), zip_name)

        print(f'\n📦 جارٍ إنشاء ملف ZIP...')
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path in drive_files:
                zipf.write(file_path, os.path.basename(file_path))

        print(f'✅ تم إنشاء: {zip_name}')
        print(f'   المسار: {zip_path}')

        # Try to download
        print(f'\n📥 جارٍ تحميل ملف ZIP...')
        try:
            files.download(zip_path)
            print(f'✅ تم بدء التحميل!')
        except Exception as e:
            print(f'⚠ تعذر التحميل التلقائي: {e}')
            print(f'\n📌 يمكنك تحميل الملفات يدوياً من Google Drive:')
            print(f'   📁 المجلد: {drive_output_dir}')
            print(f'   📦 ملف ZIP: {zip_path}')

    else:
        print('\n⚠ لم يتم العثور على ملفات للتحميل')

else:
    # No Google Drive - download directly
    print('\nجارٍ إعادة تسمية الملفات...')

    generated_files = []
    if 'txt' in output_formats:
        generated_files.extend(glob.glob(f"{output_dir}/*.txt"))
    if 'srt' in output_formats:
        generated_files.extend(glob.glob(f"{output_dir}/*.srt"))

    for file_path in generated_files:
        file_name = os.path.basename(file_path)
        base, ext = os.path.splitext(file_name)

        if base in video_titles:
            title = video_titles[base]
            safe_title = re.sub(r'[\\/:*?"<>|.]', '', title).strip()

            if not safe_title:
                continue

            new_path = os.path.join(output_dir, f"{safe_title}{ext}")

            counter = 1
            while os.path.exists(new_path):
                new_path = os.path.join(output_dir, f"{safe_title}_{counter}{ext}")
                counter += 1

            os.rename(file_path, new_path)

    # Download all files
    print('جارٍ تنزيل الملفات النصية.')

    final_files = []
    if 'txt' in output_formats:
        final_files.extend(glob.glob(f"{output_dir}/*.txt"))
    if 'srt' in output_formats:
        final_files.extend(glob.glob(f"{output_dir}/*.srt"))

    archive_path = os.path.join(output_dir, 'archive.txt')
    all_files = [f for f in final_files if f != archive_path]

    if len(all_files) > 4:
        # Create ZIP for many files
        print(f"📦 عدد الملفات ({len(all_files)}) - جارٍ إنشاء ملف ZIP...")

        zip_path = os.path.join(output_dir, 'all_transcripts.zip')
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for file_path in all_files:
                zipf.write(file_path, os.path.basename(file_path))

        print(f"✅ تم إنشاء: all_transcripts.zip")
        files.download(zip_path)
        print(f"✅ تم بدء التحميل: all_transcripts.zip")

    else:
        # Download files individually
        for file_path in all_files:
            files.download(file_path)

print('\n🎉 انتهى!')